In [1]:
import h3pandas
import geopandas as gpd
import matplotlib as plt
import os 
import pandas as pd
import folium
pd.set_option('display.max_columns', None)
import numpy as np

import sys
sys.path.append("../")
import src.supportCharts as sch

sys.path.append("../")
import src.supportClean as sc

sys.path.append("../")
import src.bibliotheque as bb

sys.path.append("../")
import src.supportExtract as se

In [2]:
path = '../output/geojson/'

In [3]:
test = se.openJSONandGetH3()
test

...opening airbnb...
done!
...opening fotocasa...
done!
...opening catastro...
done!
...opening locales...
done!
...opening tripadvisor...


,host_listings_count,beds,abnb_tot_price,number_of_reviews,id,reviews_per_month,geometry,name_clean,min_price_fc,max_price_fc,med_price_fc,price_clean_fc,conditionOfConstruction_clean_hotels,conditionOfConstruction_clean_houses,parcs_hotels,parcs_houses,srf_hotels,srf_houses,agrupaciones,alcohol,alojamiento,alquillercoche,desconocidos,educacion,fiesta,locutoris,noalim,pastelerias,prox_fresco,restaurants,sex,shops24h,supers,supersbig,vacios,name,tp_pricerange_0,tp_pricerange_1,tp_pricerange_2,tp_pricerange_3
h3_10,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8a390c8499a7fff,5.0,10.0,150150,48.0,2.0,0.67,"POLYGON ((-3.85175 40.46995, -3.85257 40.46970...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8a390c84d227fff,1.0,2.0,150,0.0,1.0,0.00,"POLYGON ((-3.85188 40.47858, -3.85271 40.47832...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8a390c84d2affff,1.0,3.0,150,1.0,1.0,0.81,"POLYGON ((-3.85164 40.47532, -3.85246 40.47506...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8a390c84d2e7fff,6.0,6.0,3528,279.0,2.0,3.72,"POLYGON ((-3.84917 40.47610, -3.84999 40.47584...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8a390c86bb4ffff,1.0,1.0,39,0.0,1.0,0.00,"POLYGON ((-3.87162 40.50575, -3.87244 40.50550...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8a392b784ceffff,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-3.51611 40.69073, -3.51694 40.69048...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0
8a392b7911a7fff,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-3.51297 40.63121, -3.51380 40.63095...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0
8a392b7918effff,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-3.50884 40.63714, -3.50966 40.63688...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,3.0,0.0,0.0


In [ ]:
test_1 = (gpd.read_file('../output/geojson/airbnb.geojson').assign(count=1).h3.geo_to_h3_aggregate(10, operation = bb.airbnb_operation, 
                         lat_col = 'latitude', lng_col = 'longitude'))
test_1

In [ ]:
gpd.read_file('../output/geojson/fotocasa.geojson')

In [ ]:
test_2 = (gpd.read_file('../output/geojson/fotocasa.geojson')
    .assign(count=1)
    .h3.geo_to_h3_aggregate(10, operation = bb.fotocasa_operation, 
        lat_col = 'latitude', lng_col = 'longitude'))
test_2

In [ ]:
polygon = pd.concat([test_1, test_2], axis = 0)
polygon['longitude'] = polygon.centroid.x  
polygon['latitude'] = polygon.centroid.y
polygon.to_file('../output/mapped/grid_competencia_suelo_test.geojson')

In [ ]:
gdf = sc.openMultipleGeospatial(path)
print(gdf.shape)
gdf.head()

In [ ]:
pd.read_csv('../data/grid.csv')

In [ ]:
type(gdf)

In [ ]:
gdf['price'].value_counts()

In [ ]:
gdf = sc.convertColumnsToNumeric(gdf)

# generating hexagons

In [ ]:
APERTURE_SIZE = 10

## point hexagons

In [ ]:
gdf = gpd.GeoDataFrame(gdf, geometry=gpd.points_from_xy(gdf.longitude, gdf.latitude)).set_crs('epsg:4326')

In [ ]:
sch.plotMap((gdf#.drop(['lng'], axis = 1)
 .select_dtypes(include=np.number)
 .assign(count=1)
 .h3.geo_to_h3_aggregate(APERTURE_SIZE, operation = 'sum', lat_col = 'latitude', lng_col = 'longitude')).h3.h3_to_geo_boundary(), 'number_of_reviews')

In [ ]:
dfh = (gdf.fillna(0)
 .assign(count=1)
 .h3.geo_to_h3_aggregate(APERTURE_SIZE, operation = bb.h3_operations, 
                         lat_col = 'latitude', lng_col = 'longitude'))#.reset_index()

In [ ]:
dfh_polygon = dfh
dfh_polygon.head()

In [ ]:
dfh_polygon.rename(bb.rename_dict, axis = 1, inplace = True)

In [ ]:
dfh_polygon['longitude'] = dfh_polygon.centroid.x  
dfh_polygon['latitude'] = dfh_polygon.centroid.y

In [ ]:
dfh_polygon.head()

In [ ]:
sch.plotMap(dfh_polygon, 'local200_n_vacios')

In [ ]:
#sch.foliumMap(dfh_polygon, 'polygon')

In [ ]:
#sc.exportFiles(test, test, 'testMapas')

In [ ]:
sc.exportFiles(dfh[['fc_tot_offer', 'airbnb_tot_offer', 'abnb_tot_price', 'price_clean_fc', 'geometry', 'latitude', 'longitude']], 
    dfh[['fc_tot_offer', 'airbnb_tot_offer', 'abnb_tot_price', 'price_clean_fc', 'latitude', 'longitude']], 'grid_competencia_suelo')